# Markov Model with Linear Regression

Importing libraries

In [11]:
import pandas as pd
import numpy as np
import json
from sklearn.linear_model import LinearRegression

Importing the data

In [12]:
with open('../data/raw_data.json') as f:
    data = json.load(f)

Setting global variables:
- where to cut data
- what bin size to use (in ms)
- if data should be shifted across spots

In [13]:
cut_start = 1734519960000 # 12:06
cut_end = 1734520799000 # 12:19:59
binsize = 15000 # ms = 15s
shift = 1
outfile_model = "../data-markov/probabilities-15s-shift-1.csv"
outfile_metrics = "../data-markov/markov_metrics_long.csv"

General helper functions to cut and bin the data

In [14]:
# cut data
def cut_data(data, cut_start, cut_end):
    return [timestamp for timestamp in data if timestamp >= cut_start and timestamp <= cut_end]

# count values in bins
def bin_data(data, start, end, binsize):
    bins = np.zeros(int((end - start) / binsize) + 1)
    for timestamp in data:
        bins[int((timestamp - start) / binsize)] += 1
    return bins

Preprocess the data by cutting to start and end

In [15]:
for spot in data["18. Dec (Wednesday)"]:
    if spot == "Metadata":
        continue
    data["18. Dec (Wednesday)"][spot] = cut_data(data["18. Dec (Wednesday)"][spot], cut_start, cut_end)

d = data["18. Dec (Wednesday)"]

Helper function for the regression and model

In [16]:
def get_flow_rate(data1, data2):
    regr = LinearRegression().fit(data1, data2)
    return regr.coef_[0][0]

# format data for regression
def get_regr_data(spot1, spot2, shift=0):
    # if shift present, cut end from to_state, start from end_state so that s1(ti) -> s2(ti+shift)
    data1 = spot1[:-shift] if shift > 0 else spot1
    data2 = spot2[shift:]

    # y is the combined vectors of both vectors
    y = []
    y.append(data1)
    y.append(data2)
    y = np.array(y)
    y.flatten()
    y = np.array(y).reshape(-1,1)

    # x is the identifiers, 0 for the first spot, 1 for the second spot
    x = [0] * len(data1) + [1] * len(data2)
    x = np.array(x).reshape(-1,1)
    return x, y

# returns sum of masses at specified spots
def metric_summed(pop, end_spots, spots):
    positions = []
    for end_spot in end_spots:
        positions.append(spots.index(end_spot))
    metric_sum = 0
    for pos in positions:
        metric_sum += pop[pos]
    return metric_sum

# simulate a markov model by matrix multiplication
# can also simulate on a list of probabilities by setting type to any value except simple
def simulate_markov(initial_pop, transition_matrix, n=1, type="simple"):
    crt_pop = initial_pop
    for i in range(n):
        # if only one transition matrix is given
        if type == "simple":
            crt_pop = np.dot(crt_pop, transition_matrix)
        # if list of matrices is given (simulate over time)
        else: 
            crt_pop = np.dot(crt_pop, transition_matrix[i])
    return crt_pop

Main function to generate a markov model

In [17]:
def get_regression_model(d, spots, binsize, start, end, guide_dict, shift=0):
    d_binned = {}
    for spot, timestamps in d.items():
        if spot not in spots:
            continue
        binned_data = bin_data(timestamps, start, end, binsize)
        d_binned[spot] = binned_data
    
    M = np.zeros((len(spots),len(spots)))

    # for each row
    for i, spot1 in enumerate(spots):
        # assign pass probabilities, or 0
        for j, spot2 in enumerate(spots):
            # if current spot is allowed
            if guide_dict[spot1][j] != "None":
                x, y = get_regr_data(d_binned[spot1], d_binned[spot2], shift=shift)
                # negative flow rate -> queue, vice versa
                M[i][j] = get_flow_rate(x, y)
            else: 
                M[i][j] = - np.inf
        
        flowrates = [x for x in M[i] if x != - np.inf]
        min_flow = min(flowrates)
        max_flow = max(flowrates)
        uniform_probability = 1 / len(flowrates)


        # transform to [0,1] or set to 0
        for j, spot2 in enumerate(spots):
            if M[i][j] != - np.inf:
                if max_flow != min_flow:
                    # [0,1] scaling approach
                    flow_factor = ((M[i][j] - min_flow) / (max_flow - min_flow))
                    M[i][j] = flow_factor
                else:
                    M[i][j] = uniform_probability
            else:
                M[i][j] = 0
        # cleanup
        sum_of_transitions = np.sum(M[i])
        if sum_of_transitions != 1:
                for j in range(len(spots)):
                    M[i][j] = M[i][j] / sum_of_transitions
    return M

Setting parameters for the markov model
- Guide dict is a dictionary containing all legal transitions
- spots is a list of named spots in guide_dict and data to consider for the analysis
- end spots is a subset of spots, and are considered the final spots for the metrics later

In [18]:
guide_dict = {'Entrance_R':     ["Pass", "None", "None", "Pass", "Pass", "None", "None", "Pass", "None", "None"],
              'Entrance_L':     ["None", "Pass", "None", "Pass", "Pass", "None", "None", "Pass", "None", "None"],
              'Cutlery':        ["Pass", "Pass", "Pass", "None", "None", "None", "None", "None", "None", "None"],
              'Auswahl':        ["None", "None", "None", "Pass", "None", "Pass", "Pass", "None", "Pass", "Pass"],
              'Day_Menu':       ["None", "None", "None", "None", "Pass", "Pass", "Pass", "None", "Pass", "Pass"],
              'Cash_T':         ["None", "None", "None", "None", "None", "Pass", "None", "None", "None", "None"],
              'Cash_B':         ["None", "None", "None", "None", "None", "None", "Pass", "None", "None", "None"],
              'Veggie':         ["None", "None", "None", "None", "None", "Pass", "Pass", "Pass", "Pass", "Pass"],
              'Veggie_Cash_R':  ["None", "None", "None", "None", "None", "None", "None", "None", "Pass", "None"],
              'Veggie_Cash_L':  ["None", "None", "None", "None", "None", "None", "None", "None", "None", "Pass"]
}

spots = ['Entrance_R','Entrance_L','Cutlery','Auswahl','Day_Menu','Cash_T', 'Cash_B','Veggie','Veggie_Cash_R','Veggie_Cash_L']
end_spots = ['Cash_T', 'Cash_B','Veggie_Cash_R','Veggie_Cash_L']

Getting the markov model, and saving its probability matrix to a file

In [19]:
regr_model = get_regression_model(d, spots, binsize, cut_start, cut_end, guide_dict, shift)
pd.DataFrame(regr_model).to_csv(outfile_model, header=spots, index=spots)

Simulating and saving the results to a file in long format

In [20]:
initial_pop_all = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

changed_model = regr_model.copy()
changed_model[2] = [0.25,0.25,0.5,0,0,0,0,0,0,0]

changed_model_2 = regr_model.copy()
changed_model_2[2] = [0.5,0.5,0,0,0,0,0,0,0,0]

rdf = []
for i in range(3, 11):
    end_pops_all = simulate_markov(initial_pop_all, regr_model, n=i)
    rdf.append([str(i), "Observed", metric_summed(end_pops_all, end_spots, spots)])

    end_pops_changed = simulate_markov(initial_pop_all, changed_model, n=i)
    rdf.append([str(i), "Theoretical A", metric_summed(end_pops_changed, end_spots, spots)])

    end_pops_changed_2 = simulate_markov(initial_pop_all, changed_model_2, n=i)
    rdf.append([str(i), "Theoretical B", metric_summed(end_pops_changed_2, end_spots, spots)])

print(rdf)

pd.DataFrame(rdf).to_csv(outfile_metrics, header=["Simulations", "Model", "Metric"], index=False)

[['3', 'Observed', np.float64(0.13778818320313418)], ['3', 'Theoretical A', np.float64(0.19570928172358962)], ['3', 'Theoretical B', np.float64(0.39141856344717924)], ['4', 'Observed', np.float64(0.334294536450545)], ['4', 'Theoretical A', np.float64(0.4168587259600049)], ['4', 'Theoretical B', np.float64(0.6380081701964203)], ['5', 'Observed', np.float64(0.5228151776050011)], ['5', 'Theoretical A', np.float64(0.6026249189150706)], ['5', 'Theoretical B', np.float64(0.7883911118701364)], ['6', 'Observed', np.float64(0.6748860108705194)], ['6', 'Theoretical A', np.float64(0.7399490265164741)], ['6', 'Theoretical B', np.float64(0.8772731341178776)], ['7', 'Observed', np.float64(0.7862654427836943)], ['7', 'Theoretical A', np.float64(0.8344839773964768)], ['7', 'Theoretical B', np.float64(0.9290189282764798)], ['8', 'Observed', np.float64(0.8630622720317846)], ['8', 'Theoretical A', np.float64(0.8967163151609135)], ['8', 'Theoretical B', np.float64(0.9589486529253503)], ['9', 'Observed', n